In [1]:
# Import useful libraries
import ee, ipywidgets, datetime, folium, utils, concurrent.futures, sentinel_satellites
from folium.plugins import Draw

# Manure date detector 🌱
The objective of this notebook is to create an interactive application that can allow to obtain (in a simple manner) when a crop field have been manured (it works also with multiple crops).

This notebooks gathers all the results and observations that have been understood in a previous phase.

## Select the crop field/s of interest
In the following line, through the interactive map, you can draw a polygon that represent your crop field of interest (it works also if you select multiple crops).<br>
Once all the crop fields have been selected, you just need to click on the **Export** button that you can find on the upper right part of the map. <br>
**Please add the new created JSON file inside the tests folder (within the same folder where you also have this notebook).**

In [2]:
# Create a map centered at a certain location
m = folium.Map(tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}', attr='Tiles &copy; Esri')

# Add the drawing tool to the map
draw = Draw(export=True, filename='test-fields.json')
draw.add_to(m)

# Display the map
m

## Select dates
Through the following widgets you have to choose within which range of dates you want to check whether the crop fields have been manured.

In [3]:
start_date_widget = ipywidgets.widgets.DatePicker(description='Start date', value=(datetime.date.today() - datetime.timedelta(days=365)), disabled=False)
display(start_date_widget)

end_date_widget = ipywidgets.widgets.DatePicker(description='End date', value=datetime.date.today(), disabled=False)
display(end_date_widget)

DatePicker(value=datetime.date(2022, 4, 4), description='Start date')

DatePicker(value=datetime.date(2023, 4, 4), description='End date')

In [4]:
fields_df = utils.get_fields_df('./test-fields.json')
fields_df

,crop_field_name,polygon_coordinates
0,P-Test1,"[(-74.870903, 45.550196), (-74.870088, 45.5487..."
1,P-Test2,"[(-2.806106, 51.990206), (-2.806556, 51.98969)..."
2,P-Test3,"[(32.237776, 31.053996), (32.237883, 31.052782..."


## Extracting features from the crop-field/s specified
Now the objective is to gather useful features that will be further used in order to understand when the selected crop fields have been manured, within the specified range of dates.<br>
In order to extract efficiently the features, [sentinel-satellites](https://pypi.org/project/sentinel-satellites/) PyPI library has been used. Read the documentation in order to choose correctly the `fields_thread` parameter value.<br>
Since the mentioned library relies on Google Earth Engine APIs you have to authenticate to GEE with your account (if you don't already have an account go at the [earth-engine website](https://earthengine.google.com/)).

In [5]:
# Initializes the Google Earth Engine APIs
ee.Authenticate()
ee.Initialize()


Successfully saved authorization token.


In [6]:
# Select the fields thread widget
fields_thread_widget = ipywidgets.widgets.IntSlider(value=1, min=1, max=concurrent.futures.ThreadPoolExecutor()._max_workers, step=1, description='FThreads:')
fields_thread_widget

IntSlider(value=1, description='FThreads:', max=12, min=1)

In [7]:
# Sentinel-2 features extraction
s2_features_extracted_df = sentinel_satellites.get_features(fields_df, start_date_widget.value, end_date_widget.value, sentinel=2, filters_params=['40'], fields_threads=fields_thread_widget.value).reset_index(drop=True)
s2_features_extracted_df

,crop_field_name,s2_acquisition_date,B1,B2,B3,B4,B5,B6,B7,B8,...,NDRE3,CARI1,CARI2,MCARI,MCARI1,MCARI2,BSI,GLI,ALTERATION,SDI
0,P-Test1,2022-04-05,1189.039664,1312.362017,1443.586891,1672.650084,1799.753950,1890.991933,2014.892101,2171.778824,...,0.037473,3630.998052,5305.115039,60.116022,361.406803,0.104018,7905.945397,-0.016661,1.163594,-190.185210
1,P-Test1,2022-04-18,1488.123361,1626.064874,1751.851765,1920.591261,2039.634622,2172.492101,2300.345546,2484.383866,...,0.038464,4118.313496,5473.313351,65.297905,548.627738,0.138032,9131.190256,-0.006092,1.125889,-268.402353
2,P-Test1,2022-04-28,1527.166387,1656.459496,1778.067563,1914.838319,2062.850756,2292.512605,2439.103866,2637.351933,...,0.039052,4197.316449,5360.686163,98.094194,827.057224,0.196014,9277.012731,-0.002127,1.141392,-50.157311
3,P-Test1,2022-04-30,1664.179496,1736.927395,1938.363697,2093.418487,2292.202689,2548.906555,2700.604370,2885.379496,...,0.033078,4687.860532,6552.340241,140.172424,898.538380,0.194648,10236.498740,0.006018,1.141514,-198.201008
4,P-Test1,2022-05-05,1675.173109,1811.020168,1968.865210,2066.161345,2285.364370,2692.398992,2879.463866,3073.797311,...,0.032643,4710.159655,6008.100809,172.443262,1299.213822,0.264192,10471.588441,0.007748,1.145418,0.913613
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134,P-Test3,2023-02-27,1076.130590,1235.583184,1494.364937,1317.355993,1818.379249,4100.243292,4910.520572,5085.416816,...,0.017497,4208.688781,3190.658200,602.125861,5702.141538,0.700830,9859.415203,0.078639,1.364717,3459.239714
135,P-Test3,2023-03-04,1245.520572,1301.679785,1565.685152,1401.885510,1904.771020,3967.880143,4662.787120,4790.475850,...,0.013507,4367.888427,3534.086501,591.136404,5135.097531,0.669996,9767.452619,0.073318,1.374669,3137.894454
136,P-Test3,2023-03-09,1213.107335,1324.075134,1588.064401,1401.860465,1923.726297,4059.343470,4750.720930,4882.783542,...,0.013709,4438.483360,3485.620967,624.015530,5303.007370,0.678823,9922.386834,0.076277,1.354816,3176.280859
137,P-Test3,2023-03-19,1734.456172,2505.665474,2343.347048,1911.236136,2371.838998,4327.341682,5034.062612,5212.105546,...,0.017377,5304.855515,2494.336384,564.535249,5427.344973,0.650841,12200.173260,0.029636,1.371566,3338.740608


In [8]:
# Sentinel-1 features extraction
s1_features_extracted_df = sentinel_satellites.get_features(fields_df, start_date_widget.value, end_date_widget.value, sentinel=1, filters_params=['ASCENDING'], fields_threads=fields_thread_widget.value).reset_index(drop=True)
s1_features_extracted_df

,crop_field_name,s1_acquisition_date,VV,VH,AVE,DIF,RAT1,RAT2,NDI,RVI
0,P-Test1,2022-04-04,-9.789369,-17.762515,-13.775942,7.973145,0.551125,1.814470,-0.289387,2.578773
1,P-Test1,2022-04-28,-12.999898,-21.546662,-17.273280,8.546764,0.603337,1.657449,-0.247398,2.494797
2,P-Test1,2022-05-10,-14.320252,-22.674035,-18.497143,8.353782,0.631571,1.583354,-0.225813,2.451626
3,P-Test1,2022-05-22,-10.074470,-17.482834,-13.778652,7.408365,0.576249,1.735360,-0.268835,2.537670
4,P-Test1,2022-06-03,-12.340505,-19.935462,-16.137983,7.594957,0.619023,1.615449,-0.235313,2.470626
...,...,...,...,...,...,...,...,...,...,...
133,P-Test3,2023-03-08,-7.323451,-15.156486,-11.239969,7.833036,0.483189,2.069583,-0.348446,2.696891
134,P-Test3,2023-03-15,-8.893600,-14.213918,-11.553759,5.320318,0.625697,1.598219,-0.230242,2.460484
135,P-Test3,2023-03-20,-8.243633,-15.861063,-12.052348,7.617429,0.519740,1.924038,-0.316014,2.632029
136,P-Test3,2023-03-27,-7.190662,-14.413658,-10.802160,7.222997,0.498878,2.004497,-0.334331,2.668662


## Detect manure dates, using the best selected ML model (pre-trained)